In [ ]:
import math
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple, Dict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from gan_synthesis.datasets.dataset import Dataset
from gan_synthesis.u_net_models.unet import UNet


@dataclass
class TrainConfig:
    seed: int = 42
    epochs: int = 200
    batch_size: int = 16
    lr: float = 5e-3
    weight_decay: float = 1e-4
    num_workers: int = 4
    pin_memory: bool = True
    amp: bool = True if torch.cuda.get_device_name(0) != 'NVIDIA GeForce GTX 1660 SUPER' else False # Training on local GPU, 16 series GPUs don't play well with amp
    ckpt_dir: str = "checkpoints"
    val_split: float = 0.05
    anchor: int = 32
    lr_patience: int = 10  # ReduceLROnPlateau patience
    lr_factor: float = 0.5
    min_lr: float = 1e-5


def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True


def accuracy_from_logits(logits: torch.Tensor, targets: torch.Tensor) -> float:
    """Pixel accuracy for segmentation."""
    pred = logits.argmax(dim=1)
    correct = (pred == targets).sum().item()
    total = targets.numel()
    return correct / total


def build(cfg: TrainConfig):
    """Constructing everything needed for training/eval"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- Data ---
    dataset = Dataset()
    train_set, val_set = dataset.split(1 - cfg.val_split)

    common_loader_kwargs = dict(
        batch_size=cfg.batch_size,
        num_workers=cfg.num_workers,
        pin_memory=cfg.pin_memory,
        persistent_workers=cfg.num_workers > 0,
    )
    train_loader = DataLoader(train_set, shuffle=True,  **common_loader_kwargs)
    val_loader   = DataLoader(val_set,   shuffle=False, **common_loader_kwargs)

    # Model + Loss + Optimizer + Scheduler +  Scaler
    model = UNet(anchor=cfg.anchor).to(device) 

    class_weights = getattr(dataset, "weights", None)
    if class_weights is not None:
        class_weights = class_weights.to(device=device, dtype=torch.float32)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=cfg.lr_factor,
        patience=cfg.lr_patience,
        min_lr=cfg.min_lr,
    )
    scaler = torch.amp.GradScaler("cuda") if (cfg.amp and device.type == "cuda") else None

    return {
        "device":       device,
        "model":        model,
        "criterion":    criterion,
        "optimizer":    optimizer,
        "scheduler":    scheduler,
        "scaler":       scaler,
        "train_set":    train_set,
        "val_set":      val_set,
        "train_loader": train_loader,
        "val_loader":   val_loader,
    }


def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    scaler: torch.amp.GradScaler | None,
    device: torch.device,
) -> Dict[str, float]:
    model.train()
    running_loss = 0.0
    running_acc = 0.0

    pbar = tqdm(loader, desc="Train", leave=False)
    for images, seg in pbar:
        images = images.to(device=device, dtype=torch.float32, non_blocking=True)
        targets = seg.squeeze(1).long().to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        if scaler is not None:
            with torch.amp.autocast(device_type=device.type, enabled=(scaler is not None)):
                logits = model(images)
                loss = criterion(logits, targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            logits = model(images)
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

        acc = accuracy_from_logits(logits.detach(), targets)
        running_loss += loss.item()
        running_acc += acc
        pbar.set_postfix({"loss": f"{loss.item():.4f}", "acc": f"{acc:.4f}"})

    n = len(loader)
    return {"loss": running_loss / n, "acc": running_acc / n}


@torch.no_grad()
def evaluate(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
) -> Dict[str, float]:
    model.eval()
    running_loss = 0.0
    running_acc = 0.0

    with torch.inference_mode():
        pbar = tqdm(loader, desc="Val", leave=False)
        for images, seg in pbar:
            images = images.to(device=device, dtype=torch.float32, non_blocking=True)
            targets = seg.squeeze(1).long().to(device, non_blocking=True)
            logits = model(images)
            loss = criterion(logits, targets)
            acc = accuracy_from_logits(logits, targets)

            running_loss += loss.item()
            running_acc += acc
            pbar.set_postfix({"loss": f"{loss.item():.4f}", "acc": f"{acc:.4f}"})

    n = len(loader)
    return {"loss": running_loss / n, "acc": running_acc / n}


def main(cfg: TrainConfig) -> Tuple[nn.Module, Dict]:
    set_seed(cfg.seed)
    parts = build(cfg)

    device       = parts["device"]
    model        = parts["model"]
    criterion    = parts["criterion"]
    optimizer    = parts["optimizer"]
    scheduler    = parts["scheduler"]
    scaler       = parts["scaler"]
    train_loader = parts["train_loader"]
    val_loader   = parts["val_loader"]

    # --- Bookkeeping ---
    Path(cfg.ckpt_dir).mkdir(parents=True, exist_ok=True)
    best_val = math.inf
    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    writer = SummaryWriter(log_dir=str(Path("runs") / "exp1"))  # <<< add

    for epoch in range(cfg.epochs):
        print(f"\nEpoch {epoch+1}/{cfg.epochs}")

        train_metrics = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device)
        val_metrics   = evaluate(model, val_loader, criterion, device)

        scheduler.step(val_metrics["loss"])

        history["train_loss"].append(train_metrics["loss"])
        history["train_acc"].append(train_metrics["acc"])
        history["val_loss"].append(val_metrics["loss"])
        history["val_acc"].append(val_metrics["acc"])

        # TensorBoard scalars
        writer.add_scalars("Loss", {"train": train_metrics["loss"],
                                    "val":   val_metrics["loss"]}, epoch)

        writer.add_scalars("Accuracy", {"train": train_metrics["acc"],
                                        "val":   val_metrics["acc"]}, epoch)
        writer.flush()

        print(
            f"train: loss {train_metrics['loss']:.4f} | acc {train_metrics['acc']:.4f} "
            f"|| val: loss {val_metrics['loss']:.4f} | acc {val_metrics['acc']:.4f} "
            f"|| lr: {optimizer.param_groups[0]['lr']:.2e}"
        )

        # Save latest
        torch.save(
            {
                "epoch": epoch + 1,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "scaler_state": scaler.state_dict() if scaler is not None else None,
                "cfg": cfg.__dict__,
                "history": history,
            },
            Path(cfg.ckpt_dir) / "latest.pt",
        )

        # Save best on val loss
        if val_metrics["loss"] < best_val:
            best_val = val_metrics["loss"]
            torch.save(model.state_dict(), Path(cfg.ckpt_dir) / "best_model.pt")

    print("\nTraining complete.")
    writer.close()
    return model, history 

   

if __name__ == "__main__":
    cfg = TrainConfig()
    model, history = main(cfg)




Epoch 1/200


train: loss 0.9589 | acc 0.7130 || val: loss 0.7903 | acc 0.7123 || lr: 5.00e-03

Epoch 2/200


train: loss 0.6900 | acc 0.7669 || val: loss 0.6601 | acc 0.7772 || lr: 5.00e-03

Epoch 3/200


train: loss 0.5903 | acc 0.7965 || val: loss 0.5623 | acc 0.8091 || lr: 5.00e-03

Epoch 4/200


train: loss 0.5592 | acc 0.8063 || val: loss 0.5319 | acc 0.8144 || lr: 5.00e-03

Epoch 5/200


...

train: loss 0.2758 | acc 0.9025 || val: loss 0.2990 | acc 0.8848 || lr: 1.00e-05

Epoch 197/200


train: loss 0.2762 | acc 0.9024 || val: loss 0.2994 | acc 0.8848 || lr: 1.00e-05

Epoch 198/200


train: loss 0.2759 | acc 0.9023 || val: loss 0.2995 | acc 0.8839 || lr: 1.00e-05

Epoch 199/200


train: loss 0.2759 | acc 0.9022 || val: loss 0.2997 | acc 0.8843 || lr: 1.00e-05

Epoch 200/200


train: loss 0.2760 | acc 0.9024 || val: loss 0.2994 | acc 0.8843 || lr: 1.00e-05

Training complete.
